In [1]:
import sys 
sys.path.append(r"C:\Users\a-sochat\development\Qcodes_Sohail_Clone\Qcodes")

import numpy as np

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter
from qcodes.sweep.sweep import (
    Nest, Zip, Chain, ParameterSweep, ParameterWrapper, FunctionSweep, FunctionWrapper, measurement, setter
)

C:\Users\a-sochat\AppData\Local\Continuum\Anaconda3\envs\qcodes\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [18]:
class Printer:
    def __init__(self, sweep_object): 
        self._ind, self._dep = sweep_object.parameter_table.flatten()
        self._symbols_list = sweep_object.parameter_table.symbols_list()
        
    def __enter__(self): 
        header_ind = "\t".join(["{} [{}]".format(*i) for i in self._ind.items()])
        header_dep = "\t".join(["{} [{}]".format(*i) for i in self._dep.items()])
        sep = " | "
        print((header_ind + sep + header_dep).strip(sep))
        
        return self
    
    def __exit__(self, type, value, traceback): 
        pass

    def __call__(self, result):
        print(" " + "\t ".join([str(result[ip]) for ip in self._symbols_list]))

# Lets introduce a basic sweep object 

In [19]:
x = ManualParameter("x", unit="V")

In [20]:
sweep_object = ParameterSweep(x, lambda: [0, 1])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

x [V]
 0
 1


We have generated a small 1D coordinate layout with size 2 

# How do we make a 2D sweep? 

In [21]:
y = ManualParameter("y", unit="V")

In [22]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])
sweep_object = Nest([soy, sox])

In [23]:
with Printer(sweep_object) as printer:
    for i in sweep_object:  # X is the inner axis : 
        printer(i)

y [V]	x [V]
 0	 0
 0	 1
 1	 0
 1	 1


This represents a 2D layout of 2x2 

# We can extend this to ND

In [24]:
z = ManualParameter("z", unit="V")

In [25]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])
soz = ParameterSweep(z, lambda: [0, 1])
sweep_object = Nest([soz, soy, sox])

In [26]:
with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

z [V]	y [V]	x [V]
 0	 0	 0
 0	 0	 1
 0	 1	 0
 0	 1	 1
 1	 0	 0
 1	 0	 1
 1	 1	 0
 1	 1	 1


We have created a 2x2x2 layout 

# This is how we can perform a measurement  

In [27]:
m = ManualParameter("m", unit="A")
m.get = lambda: x() ** 2 + y()

In [28]:
x(3)
y(1)
sweep_object = ParameterWrapper(m)

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

m [A]
 10


Wrapping a parameter in the "ParameterWrapper" class makes a sweep object which iterates once and returns the "get" value of the parameter. We can use this to create looped measurements. 

In [29]:
sox = ParameterSweep(x, lambda: [0, 1, 2])
soy = ParameterSweep(y, lambda: [0, 1, 3, 4])
meas = ParameterWrapper(m)
sweep_object =  Nest([soy, sox, meas])

In [32]:
with Printer(sweep_object) as printer:
    for i in  sweep_object: 
        printer(i)

y [V]	x [V] | m [A]
 0	 0	 0
 0	 1	 1
 0	 2	 4
 1	 0	 1
 1	 1	 2
 1	 2	 5
 3	 0	 3
 3	 1	 4
 3	 2	 7
 4	 0	 4
 4	 1	 5
 4	 2	 8


We see a seperator "|" before "m" in the header. This means that "m" is a dependent parameter, depending on both "x" and "y". In fact, everything after the "|" seperator is a dependent parameter

In [33]:
n = ManualParameter("n", unit="A")
n.get = lambda: x() - y() ** 2 + 16

sox = ParameterSweep(x, lambda: [0, 1, 2])
soy = ParameterSweep(y, lambda: [0, 1, 3, 4])
meas1 = ParameterWrapper(m)
meas2 = ParameterWrapper(n)
sweep_object =  Nest([soy, sox, meas1, meas2])

In [34]:
with Printer(sweep_object) as printer:
    for i in  sweep_object: 
        printer(i)

y [V]	x [V] | m [A]	n [A]
 0	 0	 0	 16
 0	 1	 1	 17
 0	 2	 4	 18
 1	 0	 1	 15
 1	 1	 2	 16
 1	 2	 5	 17
 3	 0	 3	 7
 3	 1	 4	 8
 3	 2	 7	 9
 4	 0	 4	 0
 4	 1	 5	 1
 4	 2	 8	 2


# Introducing chaining

In [37]:
x(4)
sweep_object = Nest([
    soy, 
    Chain([
        meas1, 
        Nest([sox, meas2])
    ])
])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

y [V]	x [V] | m [A]	n [A]
 0	 None	 16	 None
 0	 0	 None	 16
 0	 1	 None	 17
 0	 2	 None	 18
 1	 None	 5	 None
 1	 0	 None	 15
 1	 1	 None	 16
 1	 2	 None	 17
 3	 None	 7	 None
 3	 0	 None	 7
 3	 1	 None	 8
 3	 2	 None	 9
 4	 None	 8	 None
 4	 0	 None	 0
 4	 1	 None	 1
 4	 2	 None	 2


We have woven together a 1D and 2D loop. Notice that we immediately see that "m" only depends on y and "n" depends on x and y. There is no need to explicitly state this. With will become important when we discuss data sets. The above sweep is equivalent to...

# We can use arbitrary functions instead of parameters as measurements

In [38]:
@measurement([("meas3", "H")])
def measurement_function(): 
    return int(np.random.normal(0, 1) * 10) / 10

In [39]:
sweep_object = Nest([ParameterSweep(x, lambda: [0, 1, 2, 3]), FunctionWrapper(measurement_function)])

with Printer(sweep_object) as printer: 
    for i in sweep_object: 
        printer(i)

x [V] | meas3 [H]
 0	 -1.2
 1	 -1.6
 2	 -1.3
 3	 1.1


In [40]:
@measurement([("m3", "H"), ("m4", "H")])
def measurement_function(): 
    meas3 = int(np.random.normal(0, 1) * 10) / 10
    meas4 = int(np.random.normal(-5, 1) * 10) / 10
    return meas3, meas4

In [41]:
sweep_object = Nest([ParameterSweep(x, lambda: [0, 1, 2, 3]), FunctionWrapper(measurement_function)])

with Printer(sweep_object) as printer: 
    for i in sweep_object: 
        printer(i)

x [V] | m3 [H]	m4 [H]
 0	 -0.8	 -6.0
 1	 1.5	 -4.7
 2	 -1.9	 -5.0
 3	 -0.9	 -5.7


# We can also use functions as loop parameters

In [42]:
t = ManualParameter("z")
t.get = lambda: int(np.random.uniform(0, 100))

@setter([("xs", "V")])
def setter1(value): 
    x.set(2 * value)
    return value 

In [43]:
sweep_object = Nest([FunctionSweep(setter1, lambda: [0, 1, 2, 3]), FunctionWrapper(measurement_function)])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

xs [V] | m3 [H]	m4 [H]
 0	 0.4	 -5.5
 1	 -0.2	 -5.7
 2	 0.8	 -5.9
 3	 -0.9	 -5.1


In [44]:
@setter([("xs", "V"), ("ys", "V")])
def setter2(xv, yv):
    x.set(xv)
    y.set(yv)

In [45]:
sweep_object = Nest([
    FunctionSweep(setter2, lambda: zip([0, 1, 2, 3], [4, 5, 7, 8])), 
    FunctionWrapper(measurement_function)
])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

xs [V]	ys [V] | m3 [H]	m4 [H]
 0	 4	 -1.4	 -4.9
 1	 5	 -0.5	 -4.9
 2	 7	 0.9	 -6.0
 3	 8	 -0.2	 -5.4


# Finally, the sweep values need not be a list or an numpy array, but also a generating function

In [ ]:
class Measurement:
    def __init__(self): 
        self._value = self._roll_dice()
    def __call__(self): 
        self._value = self._roll_dice()
        return {"measurement": {"unit": "H", "value": self._value, "independent_parameter": False}}
    def _roll_dice(self): 
        return np.random.normal(0, 1, (3,))
    def value(self): 
        return self._value

measurement_function = Measurement()

# By allowing the sweep values to be a generator, we can create a feed-back loop between the sweep object and the 
# measurement.

def sweep_values(): 
    value = 0.0
    while value < 2.0:
        yield value 
        value = np.sum(measurement_function.value())  # The next step depends on the measurement

def setter(value): 
    return {"dac_channel": {"unit": "H", "value": "{:.3}".format(value), "independent_parameter": True}}

for i in Nest([FunctionSweep(setter, sweep_values), FunctionWrapper(measurement_function)]): 
    pretty_print(i)

We will loop until the sum of the measurement variables equal 2.0 or more. Since measurement values are three stochstics with a N(0, 1) distribution, the distribution of the sum is N(0, sqrt(3)). The probability of finding 2.0 or more: Z = 2/sqrt(3), which is equal to 12.51%. As the next cell shows, the expectation value of the number of iterations is therefore 8.0 iterations

In [ ]:
def f(ni): # The probability of looping exactly ni times 
    p = 0.1251
    return (1 - p)**(ni - 1) * p

np.sum([ni*f(ni) for ni in range(1, 1000)])

Lets see if this is correct 

In [ ]:
count = 0
N = 10000
for _ in range(N):
    count += len(list(Nest([FunctionSweep(setter, sweep_values), FunctionWrapper(measurement_function)])))

print(count/N)

:-)